In [1]:
import os, sys
sys.path.insert(0, "/Users/magnusax/AutoML")
from gazer import GazerMetaLearner
from gazer.optimization import param_search
from sklearn.datasets import load_digits
from scipy.stats import uniform, randint
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

Using TensorFlow backend.
/Users/magnusax/AutoML/gazer/__init__.py:16: RuntimeWarning: xgboost import failed; 'xgboost' 
        will be unavailable.
  will be unavailable.""".format(lib, alias), RuntimeWarning)


In [2]:
X, y = load_digits(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.5, random_state=0)
#y_train = to_categorical(y_train)
#y_test = to_categorical(y_test)

In [3]:
learner = GazerMetaLearner(
    method='select', 
    estimators=['neuralnet'], 
    verbose=0
)

param_grid = {
    'epochs': randint(5, 31),
    'input_units': randint(200, 1201),
    'dropout': [True, False],
    'p': uniform(0,0.7),
    'batch_size': randint(16, 129),
    'batch_norm': [False, True],
    'n_hidden': randint(2,4),   
    'decay_units': [True, False],
    'learning_rate': [1e-3 * x for x in range(1, 11)],
    'optimizer': ['adam', 'adagrad'],
    'gamma': uniform(1.5, 1.0),
}

data = {
    'train': (X_train, y_train), 
    'val': (X_test, y_test)
}

type_of_search = 'random'

n_iter = 100

name = learner.names[0]

modelfiles = ["tmp/model{}.hdf5".format(i) for i in range(1,6)]

In [4]:
config, df = param_search(learner, param_grid, data, 
                          type_of_search=type_of_search, 
                          n_iter=n_iter, name=name, 
                          modelfiles=modelfiles, top_n=5)

Widget Javascript not detected.  It may not be installed or enabled properly.


In [5]:
df

,batch_norm,batch_size,decay_units,dropout,epochs,gamma,input_units,learning_rate,n_hidden,optimizer,p,train_loss,val_loss,train_score,val_score
32,True,92,True,False,25,2.164772,718,0.003,3,adam,0.199357,0.0004,0.0531,1.0000,0.9844
20,True,50,False,True,9,2.157223,761,0.007,2,adagrad,0.406312,0.0008,0.0637,1.0000,0.9833
31,True,115,True,False,28,2.451649,892,0.003,3,adam,0.265402,0.0005,0.0575,1.0000,0.9833
7,True,65,False,False,14,1.982446,671,0.009,3,adagrad,0.666452,0.0005,0.0662,1.0000,0.9811
45,True,39,False,True,18,1.617226,359,0.005,3,adagrad,0.277546,0.0003,0.0688,1.0000,0.9811
5,True,92,True,True,25,1.670565,936,0.005,3,adam,0.346420,0.0018,0.0948,0.9989,0.9800
72,True,86,True,False,24,2.440254,452,0.003,2,adam,0.052628,0.0005,0.0623,1.0000,0.9800
61,True,83,False,False,20,2.445365,305,0.001,2,adam,0.691845,0.0006,0.0705,1.0000,0.9800
88,False,22,True,False,30,1.970324,646,0.001,3,adam,0.400966,0.0000,0.1147,1.0000,0.9800
38,True,40,False,True,30,2.372189,827,0.005,2,adagrad,0.237888,0.0000,0.0858,1.0000,0.9800


In [6]:
from keras.models import Sequential, load_model

In [7]:
model = load_model(modelfiles[0])

In [8]:
model.evaluate(X_test, to_categorical(y_test))

899/899 [==============================] - 5s 5ms/step


[0.05312484056671752, 0.98442714126807562]

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_144 (Dense)            (None, 718)               46670     
_________________________________________________________________
activation_112 (Activation)  (None, 718)               0         
_________________________________________________________________
batch_normalization_72 (Batc (None, 718)               2872      
_________________________________________________________________
dense_145 (Dense)            (None, 331)               237989    
_________________________________________________________________
activation_113 (Activation)  (None, 331)               0         
_________________________________________________________________
batch_normalization_73 (Batc (None, 331)               1324      
_________________________________________________________________
dense_146 (Dense)            (None, 153)               50796     
__________